## Part 1 - Produce Data
We elected to use the datagen connector to generate fake data for this assignment. The topic we used was 'stocktrades. The steps were as follows:
*  Open a browser and go to http://localhost:9021/
*  Select the available cluster
*  On the menu bar, select Connect
*  Click on the connect-default cluster in the Connect Clusters list.
*  Click on Add connector
*  Select DatagenConnector
*  Enter connector_stock_trades in the Name field

Then:
Generate a data stream with following configurations:
```
{
  "name": "connector_stock_trades",
  "connector.class": "io.confluent.kafka.connect.datagen.DatagenConnector",
  "key.converter": "org.apache.kafka.connect.storage.StringConverter",
  "kafka.topic": "stocktrades",
  "max.interval": "100",
  "quickstart": "Stock_Trades"
}
```


## Part 2 - Using Ksql to create at least 2 streams with filtering from topics

To begin, you need to create a stream called stocktrades with no filtering in place.

### Create JSON Stream
In order for this to play nicely with spark, we need to mimic the raw stream as a json formatted stream.

### Stream 1 - Sell Stream
It may be in the interest of the business to view only streams where the stock was sold and not bought. This would be useful in identifying which shares should be taken as a 'short' position

### Stream 2 - Buy Stream
It may also be interesting to the business to see trades that were large buys.

### Table 1 - Aggregated Buy Trades


### Table 2 - Aggregated Sell Trades

## Part 3 - Consume/Transform data with Spark Streaming

In [1]:
from pyspark.sql import SparkSession
from IPython.display import display, clear_output
import time
from pyspark.sql import functions as F
from pyspark.sql.types import StructType,StringType, StructField, IntegerType, FloatType, BinaryType

In [2]:
spark = SparkSession.builder \
        .appName('kafka') \
        .getOrCreate()

In [3]:
spark.version

'3.1.1'

In [4]:
spark._jvm.org.apache.hadoop.util.VersionInfo.getVersion()

'3.2.0'

## Raw Data Streams

In [34]:
stream_df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "broker:29092") \
  .option("startingOffsets", "earliest") \
  .option("subscribe", "STOCKTRADES_JSON") \
  .load()

In [35]:
stream_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [36]:
raw_stream = stream_df \
    .writeStream \
    .format("memory") \
    .queryName("raw_stocktrades_view") \
    .start()

In [37]:
clear_output(wait=True)
display(spark.sql('SELECT key, value FROM raw_stocktrades_view').show(20))
time.sleep(1)

+----+--------------------+
| key|               value|
+----+--------------------+
|null|[7B 22 53 49 44 4...|
|null|[7B 22 53 49 44 4...|
|null|[7B 22 53 49 44 4...|
|null|[7B 22 53 49 44 4...|
|null|[7B 22 53 49 44 4...|
|null|[7B 22 53 49 44 4...|
|null|[7B 22 53 49 44 4...|
|null|[7B 22 53 49 44 4...|
|null|[7B 22 53 49 44 4...|
|null|[7B 22 53 49 44 4...|
|null|[7B 22 53 49 44 4...|
|null|[7B 22 53 49 44 4...|
|null|[7B 22 53 49 44 4...|
|null|[7B 22 53 49 44 4...|
|null|[7B 22 53 49 44 4...|
|null|[7B 22 53 49 44 4...|
|null|[7B 22 53 49 44 4...|
|null|[7B 22 53 49 44 4...|
|null|[7B 22 53 49 44 4...|
|null|[7B 22 53 49 44 4...|
+----+--------------------+
only showing top 20 rows



None

In [38]:
raw_stream.stop()

### Convert Key Value pairs to strings

In [39]:
string_stream_df = stream_df \
    .withColumn("key", stream_df["key"].cast(StringType())) \
      .withColumn('value', stream_df["value"].cast(StringType()))

In [40]:
string_stream = string_stream_df \
    .writeStream \
    .format("memory") \
    .queryName("string_stocktrades_view") \
    .start()

In [41]:
clear_output(wait=True)
display(spark.sql('SELECT key, value FROM string_stocktrades_view').show(20))
time.sleep(1)

+----+--------------------+
| key|               value|
+----+--------------------+
|null|{"SIDE":"SELL","Q...|
|null|{"SIDE":"BUY","QU...|
|null|{"SIDE":"BUY","QU...|
|null|{"SIDE":"BUY","QU...|
|null|{"SIDE":"SELL","Q...|
|null|{"SIDE":"SELL","Q...|
|null|{"SIDE":"BUY","QU...|
|null|{"SIDE":"BUY","QU...|
|null|{"SIDE":"BUY","QU...|
|null|{"SIDE":"BUY","QU...|
|null|{"SIDE":"BUY","QU...|
|null|{"SIDE":"SELL","Q...|
|null|{"SIDE":"SELL","Q...|
|null|{"SIDE":"SELL","Q...|
|null|{"SIDE":"BUY","QU...|
|null|{"SIDE":"SELL","Q...|
|null|{"SIDE":"SELL","Q...|
|null|{"SIDE":"SELL","Q...|
|null|{"SIDE":"SELL","Q...|
|null|{"SIDE":"BUY","QU...|
+----+--------------------+
only showing top 20 rows



None

In [42]:
string_stream.stop()

## Transformation

In [43]:
schema_stocktrades =  StructType([
        StructField("SIDE", StringType(),  True),
        StructField("QUANTITY", IntegerType(),  True),
        StructField("PRICE", IntegerType(),  True),
        StructField("SYMBOL", StringType(),  True),
        StructField("ACCOUNT", StringType(), True),
         StructField("USERID", StringType(), True)
])

In [44]:
json_stream_df = string_stream_df\
    .withColumn("value", F.from_json("value", schema_stocktrades))

In [45]:
json_stream_df.printSchema()

root
 |-- key: string (nullable = true)
 |-- value: struct (nullable = true)
 |    |-- SIDE: string (nullable = true)
 |    |-- QUANTITY: integer (nullable = true)
 |    |-- PRICE: integer (nullable = true)
 |    |-- SYMBOL: string (nullable = true)
 |    |-- ACCOUNT: string (nullable = true)
 |    |-- USERID: string (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [48]:
json_stream = json_stream_df \
    .writeStream \
    .format("memory") \
    .queryName("extract_stocktrades_view") \
    .start()

In [49]:
clear_output(wait=True)
display(spark.sql('SELECT * FROM extract_stocktrades_view').show(20, False))
time.sleep(1)

+----+----------------------------------------+----------------+---------+------+-----------------------+-------------+
|key |value                                   |topic           |partition|offset|timestamp              |timestampType|
+----+----------------------------------------+----------------+---------+------+-----------------------+-------------+
|null|{SELL, 3679, 432, ZXZZT, XYZ789, User_2}|STOCKTRADES_JSON|0        |0     |2021-06-06 07:37:35.324|0            |
|null|{BUY, 4955, 73, ZTEST, ABC123, User_9}  |STOCKTRADES_JSON|0        |1     |2021-06-06 07:37:35.382|0            |
|null|{BUY, 4014, 821, ZJZZT, XYZ789, User_8} |STOCKTRADES_JSON|0        |2     |2021-06-06 07:37:35.417|0            |
|null|{BUY, 161, 928, ZVV, XYZ789, User_1}    |STOCKTRADES_JSON|0        |3     |2021-06-06 07:37:35.458|0            |
|null|{SELL, 2099, 947, ZTEST, ABC123, User_4}|STOCKTRADES_JSON|0        |4     |2021-06-06 07:37:35.503|0            |
|null|{SELL, 3337, 731, ZVV, XYZ789, Use

None

In [50]:
json_stream.stop()

### Flatten Data

In [53]:
stocktrades_stream_df = json_stream_df \
    .select( \
        F.col("key").alias("event_key"), \
        F.col("topic").alias("event_topic"), \
        F.col("timestamp").alias("event_timestamp"), \
        "value.side", \
        "value.quantity", \
        "value.price", \
        "value.symbol", \
        "value.account", \
        "value.userid"
    )

In [54]:
stocktrades_stream_df.printSchema()

root
 |-- event_key: string (nullable = true)
 |-- event_topic: string (nullable = true)
 |-- event_timestamp: timestamp (nullable = true)
 |-- SIDE: string (nullable = true)
 |-- QUANTITY: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- symbol: string (nullable = true)
 |-- account: string (nullable = true)
 |-- userid: string (nullable = true)



In [55]:
stocktrades_stream = stocktrades_stream_df \
    .writeStream \
    .format("memory") \
    .queryName("stocktrades_view") \
    .start()

In [56]:
clear_output(wait=True)
display(spark.sql('SELECT * FROM stocktrades_view').show(20))
time.sleep(1)

+---------+----------------+--------------------+----+--------+-----+------+-------+------+
|event_key|     event_topic|     event_timestamp|SIDE|QUANTITY|price|symbol|account|userid|
+---------+----------------+--------------------+----+--------+-----+------+-------+------+
|     null|STOCKTRADES_JSON|2021-06-06 07:37:...|SELL|    3679|  432| ZXZZT| XYZ789|User_2|
|     null|STOCKTRADES_JSON|2021-06-06 07:37:...| BUY|    4955|   73| ZTEST| ABC123|User_9|
|     null|STOCKTRADES_JSON|2021-06-06 07:37:...| BUY|    4014|  821| ZJZZT| XYZ789|User_8|
|     null|STOCKTRADES_JSON|2021-06-06 07:37:...| BUY|     161|  928|   ZVV| XYZ789|User_1|
|     null|STOCKTRADES_JSON|2021-06-06 07:37:...|SELL|    2099|  947| ZTEST| ABC123|User_4|
|     null|STOCKTRADES_JSON|2021-06-06 07:37:...|SELL|    3337|  731|   ZVV| XYZ789|User_1|
|     null|STOCKTRADES_JSON|2021-06-06 07:37:...| BUY|    1030|  773|  ZBZX| LMN456|User_7|
|     null|STOCKTRADES_JSON|2021-06-06 07:37:...| BUY|    2928|  136| ZTEST| ABC

None

In [57]:
stocktrades_stream.stop()

## Create neater function to generate stream
This function generates a stream from stocktrades with one line of code so its easier to call in later components

In [ ]:
def generate_stocktrades_stream():
    # Define the Spark Stream
    stream_df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "broker:29092") \
  .option("startingOffsets", "earliest") \
  .option("subscribe", "STOCKTRADES_JSON") \
  .load()
    # Convert to string types
    string_stream_df = stream_df \
    .withColumn("key", stream_df["key"].cast(StringType())) \
      .withColumn('value', stream_df["value"].cast(StringType()))
    
    # Define the Schema
    schema_stocktrades =  StructType([
        StructField("SIDE", StringType(),  True),
        StructField("QUANTITY", IntegerType(),  True),
        StructField("PRICE", IntegerType(),  True),
        StructField("SYMBOL", StringType(),  True),
        StructField("ACCOUNT", StringType(), True),
         StructField("USERID", StringType(), True)
])
    # Convert the string type to json
    json_stream_df = string_stream_df\
    .withColumn("value", F.from_json("value", schema_stocktrades))
    stocktrades_stream_df = json_stream_df \
    .select( \
        F.col("key").alias("event_key"), \
        F.col("topic").alias("event_topic"), \
        F.col("timestamp").alias("event_timestamp"), \
        "value.side", \
        "value.quantity", \
        "value.price", \
        "value.symbol", \
        "value.account", \
        "value.userid"
    )
    return stocktrades_stream_df \
    .writeStream \
    .format("memory") \
    .queryName("stocktrades_view") \
    .start()

In [ ]:
stocktrades_stream = generate_stocktrades_stream()

In [ ]:
clear_output(wait=True)
display(spark.sql('SELECT * FROM stocktrades_view').show(20))
time.sleep(1)

In [ ]:
stocktrades_stream = generate_stocktrades_stream()

In [68]:
clear_output(wait=True)
display(spark.sql('SELECT * FROM stocktrades_view').show(20))
time.sleep(1)

In [65]:
stocktrades_stream = generate_stocktrades_stream()

In [66]:
clear_output(wait=True)
display(spark.sql('SELECT * FROM stocktrades_view').show(20))
time.sleep(1)

+---------+----------------+--------------------+----+--------+-----+------+-------+------+
|event_key|     event_topic|     event_timestamp|side|quantity|price|symbol|account|userid|
+---------+----------------+--------------------+----+--------+-----+------+-------+------+
|     null|STOCKTRADES_JSON|2021-06-06 07:37:...|SELL|    3679|  432| ZXZZT| XYZ789|User_2|
|     null|STOCKTRADES_JSON|2021-06-06 07:37:...| BUY|    4955|   73| ZTEST| ABC123|User_9|
|     null|STOCKTRADES_JSON|2021-06-06 07:37:...| BUY|    4014|  821| ZJZZT| XYZ789|User_8|
|     null|STOCKTRADES_JSON|2021-06-06 07:37:...| BUY|     161|  928|   ZVV| XYZ789|User_1|
|     null|STOCKTRADES_JSON|2021-06-06 07:37:...|SELL|    2099|  947| ZTEST| ABC123|User_4|
|     null|STOCKTRADES_JSON|2021-06-06 07:37:...|SELL|    3337|  731|   ZVV| XYZ789|User_1|
|     null|STOCKTRADES_JSON|2021-06-06 07:37:...| BUY|    1030|  773|  ZBZX| LMN456|User_7|
|     null|STOCKTRADES_JSON|2021-06-06 07:37:...| BUY|    2928|  136| ZTEST| ABC

None

In [67]:
stocktrades_stream.stop()